In [3]:
from astropy import coordinates
from astropy import units as u
from astropy.coordinates import Angle, SkyCoord, EarthLocation, AltAz
from astropy.table import Table
from astropy.time import Time

from astropy.io import fits

from astroquery.alma import Alma
alma=Alma()

import numpy as np
import math

from astroplan import Observer, FixedTarget

from pytz import timezone
import pytz

alma.archive_url = 'https://almascience.eso.org'

In [4]:
source = 'J1924-2914'
baobab = 'Hauyu Baobob Liu'

In [5]:
# columns based on ObsCore model have time variables
result_ObsCore = alma.query(payload=dict(source_name_alma=source), 
                            science=False,
                            maxrec=100)

/home/Peter/Softwares/anaconda3/envs/DataQuery/lib/python3.10/site-packages/pyvo/dal/query.py:324: DALOverflowWarning: Partial result set. Potential causes MAXREC, async storage space, etc.
  warn("Partial result set. Potential causes MAXREC, async storage space, etc.",


In [6]:
#columns based on ALMA advanced query have date
result_ALMA = alma.query(payload=dict(source_name_alma=source), 
                         science=False, 
                         legacy_columns=True,
                         maxrec=100)

/home/Peter/Softwares/anaconda3/envs/DataQuery/lib/python3.10/site-packages/pyvo/dal/query.py:324: DALOverflowWarning: Partial result set. Potential causes MAXREC, async storage space, etc.
  warn("Partial result set. Potential causes MAXREC, async storage space, etc.",


In [7]:
# for convenience use first column to analysis

In [8]:
# Get source coordinate

Ra = result_ALMA['RA'][1]
Dec = result_ALMA['Dec'][1]

target_coord = SkyCoord(ra=Ra*u.deg, dec=Dec*u.deg)

In [9]:
# Get observation time information

start_time = result_ObsCore['t_min'][1]          # the time is the exact second start observation
duration_time = result_ObsCore['t_exptime'][1]

# transform into the format we can understand (UTC)
hours = int(start_time / 3600)
remaining_seconds = start_time % 3600
minutes = int(remaining_seconds / 60)
seconds = remaining_seconds - minutes * 60

obs_start_time = str(hours)+':'+str(minutes)+':'+str(seconds)

In [10]:
# get date

date = result_ALMA['Observation date'][1]
[day, month, year] = date.split('-')
obs_date = year + '-' + month + '-' + day

In [11]:
# combine time and date

obs_init_Datetime = Time(obs_date + ' ' + obs_start_time)

In [12]:
ALMA = Observer.at_site("ALMA")

In [16]:
init_PA = Angle(ALMA.parallactic_angle(obs_init_Datetime, target_coord), u.deg)

In [14]:
end_time = start_time + duration_time

hours = int(end_time / 3600)
remaining_seconds = end_time % 3600
minutes = int(remaining_seconds / 60)
seconds = remaining_seconds - minutes * 60
obs_end_time = str(hours)+':'+str(minutes)+':'+str(seconds)

obs_end_Datetime = Time(obs_date + ' ' + obs_end_time)

end_PA = Angle(ALMA.parallactic_angle(obs_end_Datetime, target_coord), u.deg)